In [ ]:
from placekey.api import PlacekeyAPI
from ast import literal_eval
import snowflake.connector
from snowflake.connector import cursor
import usaddress
import re
import pandas as pd
# from helper_functions import *
import config_final

## Dealing with Abbreviations!

In [ ]:
ctx = snowflake.connector.connect(
    user=config_final.user,
    password=config_final.password,
    account=config_final.account
)
cursor = ctx.cursor()

sql_MLS = """SELECT (CASE WHEN CC_PROPERTY_ADDR_DISPLAY_1 IS NULL THEN ADDRESS ELSE CC_PROPERTY_ADDR_DISPLAY_1 END) AS MLS_ADDRESS,
                    OWNER_FULL_NAME AS NAME,
                    CC_PROPERTY_ADDR_POSTAL_CODE AS POSTALCODE
            FROM ATTOM.MLS
            WHERE CC_PROPERTY_ADDR_CITY = 'new york' and CC_PROPERTY_ADDR_STATE='ny' """
df_MLS = pd.read_sql(sql_MLS, con=ctx)
sql_DATATREE = """SELECT SITUSFULLSTREETADDRESS AS DATATREE_ADDRESS,
SITUSHOUSENBR,SITUSDIRECTIONLEFT,SITUSSTREET,SITUSMODE,SITUSDIRECTIONRIGHT,SITUSUNITTYPE,SITUSUNITNBR,SITUSLATITUDE,SITUSLONGITUDE,
                         OWNERNAME1FULL,
                         OWNERNAME2FULL,
                         SITUSZIP5
                         
                  FROM DATATREE.ASSESSOR
                  WHERE SITUSCITY = 'new york'and SITUSSTATE='ny'  """
df_DATATREE = pd.read_sql(sql_DATATREE, con=ctx)


In [ ]:
# Drop NaN
df_MLS.dropna(subset=['MLS_ADDRESS'], inplace=True)
df_DATATREE.dropna(subset=['DATATREE_ADDRESS'], inplace=True)

# df_MLS = clean_special_char(df_MLS,'MLS_ADDRESS')
# df_DATATREE = clean_special_char(df_DATATREE,'DATATREE_ADDRESS')

df1_MLS = df_MLS.copy()
df1_DATATREE = df_DATATREE.copy()

df1_MLS['MLS_ADDRESS_Split'] = df1_MLS.apply(lambda df1_MLS : get_splited_addr(df1_MLS,'MLS_ADDRESS'), axis = 1)
df1_DATATREE['DATATREE_ADDRESS_Split'] = df1_DATATREE.apply(lambda df1_DATATREE : get_splited_addr(df1_DATATREE,'DATATREE_ADDRESS'), axis = 1)



In [ ]:
MLS_ADDRESS_final = get_comb_addr(df1_MLS,'MLS_ADDRESS_Split')
DT_ADDRESS_final = get_comb_addr(df1_DATATREE,'DATATREE_ADDRESS_Split')

# convert address format (direction, abbreviation)
MLS_ADDRESS_final,DT_ADDRESS_final=get_str_replace(MLS_ADDRESS_final,DT_ADDRESS_final)


In [ ]:
from placekey.api import PlacekeyAPI
placekey_api_key = "n1w1KOroOdSz3BVMMlg8IM4PuflcVP4V" # fill this in with your personal API key (do not share publicly)
pk_api = PlacekeyAPI(placekey_api_key)
column_map = {"DATATREE_ADDRESS" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")


In [ ]:
DT_ADDRESS_final.columns

Index(['DATATREE_ADDRESS', 'SITUSHOUSENBR', 'SITUSDIRECTIONLEFT',
       'SITUSSTREET', 'SITUSMODE', 'SITUSDIRECTIONRIGHT', 'SITUSUNITTYPE',
       'SITUSUNITNBR', 'SITUSLATITUDE', 'SITUSLONGITUDE', 'OWNERNAME1FULL',
       'OWNERNAME2FULL', 'SITUSZIP5', 'DATATREE_ADDRESS_Split',
       'AddressNumber', 'StreetNamePreDirectional', 'StreetName',
       'StreetNamePostType', 'OccupancyType', 'OccupancyIdentifier',
       'StreetNamePostDirectional', 'StreetNamePreType', 'PlaceName',
       'StateName', 'ZipCode', 'AddressNumberSuffix', 'SubaddressType',
       'SubaddressIdentifier', 'Recipient', 'BuildingName',
       'StreetNamePostModifier', 'LandmarkName', 'USPSBoxType', 'USPSBoxID'],
      dtype='object')

In [ ]:
DT_ADDRESS_final.SITUSUNITTYPE.unique()

array(['apt', '#', None, 'fl', 'unit', 'ste', 'rm', 'ph', 'lbby', 'ofc',
       'bldg', 'uppr', 'broadway', 'no', 'bsmt', 'lowr', 'off', 'slip',
       'pier'], dtype=object)

In [ ]:
DT_ADDRESS_final.OccupancyType.unique()

array(['apt', nan, 'fl', 'unit', 'ste', 'rm', 'ph', 'lbby', 'pier', 'ofc',
       'apt ste', 'uppr', 'lowr', 'fl st', 'bsmt', 'floor'], dtype=object)

In [ ]:
UnitType = DT_ADDRESS_final[['DATATREE_ADDRESS','SITUSUNITTYPE','OccupancyType']]

In [ ]:
UnitType[(UnitType['SITUSUNITTYPE']!=UnitType['OccupancyType'])&(UnitType['SITUSUNITTYPE']=='ph')]


,DATATREE_ADDRESS,SITUSUNITTYPE,OccupancyType
1610,100 w 39th st ph c,ph,NaN
1612,100 w 39th st ph e,ph,NaN
2368,230 w 56th st ph a,ph,NaN
2641,140 e 56th st ph c,ph,NaN
3338,450 w 17th st ph 1,ph,NaN
...,...,...,...
355221,415 e 54th st ph n,ph,NaN
356033,212 w 18th st ph 2,ph,NaN
356307,171 w 131st st ph 5,ph,NaN
356319,171 w 131st st ph 19,ph,NaN


## Topic: Do we want to have abbreviation transformation in the original dataset?

Conclusion: we can! But whether it's necessary to do it will depend on what we're gonna do later!

#### If unit signal exist:
For Accessor Table:\
there's no difference in the keys generated by converted/non-converted combined columns
=> we can use combined column to generate keys! it doesn't matter whether we transform those directional words/ street type or not
=> depending on what we're gonna do in the later column-bu-column string mappings! Yay!

For MLS Table:\
there's no difference in the keys generated by converted combined columns vs. original column
=> it's okay to standatdize directional words AND StreetTypes
=> but we keep the order of ['AddressNumber', 'StreetNamePreDirectional','StreetNamePreType' ,'StreetName','StreetNamePostType','StreetNamePostDirectional','OccupancyType', 'OccupancyIdentifier']

#### If unit signal does not exist:
we can substitute with the unit signal which occurs the most => 57/745 successfully generated new keys based on data in NYC

### case1: when unit signal exists

In [ ]:
def get_str_replace(MLS_ADDRESS_final,DT_ADDRESS_final):
    #need to change file path
    df = pd.read_csv('/Users/yiweihou/Downloads/Roc360/suffix_abbreviations_converted.csv')
    StreetNamePostType_dict = dict(zip(df.common_name, df.abbr))
    StreetNameDirectional_dict = {'west': 'w', 'south': 's', 'north': 'n','east': 'e'}

    DT_ADDRESS_final.replace({"StreetNamePreDirectional": StreetNameDirectional_dict},inplace=True)
    DT_ADDRESS_final.replace({"StreetNamePostType": StreetNamePostType_dict},inplace=True)

    MLS_ADDRESS_final.replace({"StreetNamePreDirectional": StreetNameDirectional_dict},inplace=True)
    MLS_ADDRESS_final.replace({"StreetNamePostType": StreetNamePostType_dict},inplace=True)
    return MLS_ADDRESS_final,DT_ADDRESS_final

if there is no abbr tranformation, i.e., we follow whatever the original data looks like

In [ ]:
test_noTraAbbr = DT_ADDRESS_final.copy()
test_noTraAbbr['ColumnA'] = test_noTraAbbr[test_noTraAbbr.columns[1:8]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

# for this step, we only test addresses with unit number!
test_DT_noTranAbbr = test_noTraAbbr[~test_noTraAbbr['SITUSUNITTYPE'].isna()][:1000]
# generate key with ColumnA-after string tranformation
# => no matter if we remove special characters or not, there is no missing info if we use ColumnA ( since we are using the original columns)
column_map = {"ColumnA" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
stad_df = get_standard_df(test_DT_noTranAbbr,column_map,"US", "new york", "ny")

df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
concat_noTranAbbr = pd.concat([df_placekey,test_DT_noTranAbbr.reset_index()],axis=1)


2022-05-06 10:46:38,749	INFO	Processed 1000 items
2022-05-06 10:46:38,750	INFO	Processed 1000 items
2022-05-06 10:46:38,750	INFO	Done


if we transform abbr

In [ ]:
# DT_ADDRESS_final[DT_ADDRESS_final['SITUSUNITTYPE']=='ph']
# since unittype and unit number is always not the same, we might need to be careful when joining splited strings together?

 
df = pd.read_csv('suffix_abbreviations_converted.csv')
StreetNamePostType_dict = dict(zip(df.common_name, df.abbr))

test_TranAbbr = DT_ADDRESS_final.copy()

StreetNameDirectional_dict = {'west': 'w', 'south': 's', 'north': 'n','east': 'e'}

test_TranAbbr.replace({"SITUSDIRECTIONLEFT": StreetNameDirectional_dict},inplace=True)
test_TranAbbr.replace({"SITUSDIRECTIONRIGHT": StreetNameDirectional_dict},inplace=True)

test_TranAbbr.replace({"SITUSMODE": StreetNamePostType_dict},inplace=True)


test_TranAbbr['ColumnA'] = test_TranAbbr[test_TranAbbr.columns[1:8]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)
test_TranAbbr = test_TranAbbr[~test_TranAbbr['SITUSUNITTYPE'].isna()][:1000]
# generate key with ColumnA-after string tranformation
# => no matter if we remove special characters or not, there is no missing info if we use ColumnA ( since we are using the original columns)
column_map = {"ColumnA" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
stad_df = get_standard_df(test_TranAbbr,column_map,"US", "new york", "ny")

df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
concat_Abbr = pd.concat([df_placekey,test_TranAbbr.reset_index()],axis=1)


2022-05-06 10:47:21,557	INFO	Processed 1000 items
2022-05-06 10:47:21,560	INFO	Processed 1000 items
2022-05-06 10:47:21,561	INFO	Done


In [ ]:
concat_noTranAbbr.head(5)

,query_id,placekey,error,index,DATATREE_ADDRESS,SITUSHOUSENBR,SITUSDIRECTIONLEFT,SITUSSTREET,SITUSMODE,SITUSDIRECTIONRIGHT,...,AddressNumberSuffix,SubaddressType,SubaddressIdentifier,Recipient,BuildingName,StreetNamePostModifier,LandmarkName,USPSBoxType,USPSBoxID,ColumnA
0,1,2p3@627-s8j-yvz,NaN,0,308 w 30th st apt 9b,308,w,30th,st,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,308 w 30th st apt 9b
1,2,NaN,Invalid address,1,138-140 w 124th st # 5a,138-140,w,124th,st,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138-140 w 124th st # 5a
2,3,2h6@627-wbn-k9f,NaN,3,345 w 14th st apt 4b,345,w,14th,st,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,345 w 14th st apt 4b
3,4,22f@627-s8j-d5f,NaN,4,7 w 14th st # 1120,7,w,14th,st,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7 w 14th st # 1120
4,5,2vj@627-s4n-2zf,NaN,5,340 e 74th st apt 5k,340,e,74th,st,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,340 e 74th st apt 5k


In [ ]:
concat_Abbr.head(5)

,query_id,placekey,error,index,DATATREE_ADDRESS,SITUSHOUSENBR,SITUSDIRECTIONLEFT,SITUSSTREET,SITUSMODE,SITUSDIRECTIONRIGHT,...,AddressNumberSuffix,SubaddressType,SubaddressIdentifier,Recipient,BuildingName,StreetNamePostModifier,LandmarkName,USPSBoxType,USPSBoxID,ColumnA
0,1,2p3@627-s8j-yvz,NaN,0,308 w 30th st apt 9b,308,w,30th,st,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,308 w 30th st apt 9b
1,2,NaN,Invalid address,1,138-140 w 124th st # 5a,138-140,w,124th,st,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138-140 w 124th st # 5a
2,3,2h6@627-wbn-k9f,NaN,3,345 w 14th st apt 4b,345,w,14th,st,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,345 w 14th st apt 4b
3,4,22f@627-s8j-d5f,NaN,4,7 w 14th st # 1120,7,w,14th,st,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7 w 14th st # 1120
4,5,2vj@627-s4n-2zf,NaN,5,340 e 74th st apt 5k,340,e,74th,st,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,340 e 74th st apt 5k


In [ ]:
# compare if abbr matters
IfAbbrMatter = pd.merge(concat_noTranAbbr[['query_id','error','placekey','ColumnA','DATATREE_ADDRESS']],concat_Abbr[['query_id','error','placekey','ColumnA']], how = 'left', right_on = ['query_id'], 
                          left_on = ['query_id'])

IfAbbrMatter[(IfAbbrMatter['placekey_x']!=IfAbbrMatter['placekey_y'])&IfAbbrMatter['placekey_x'].notna()]

# no difference

,query_id,error_x,placekey_x,ColumnA_x,DATATREE_ADDRESS,error_y,placekey_y,ColumnA_y


Conclusion: encoding directional words and streetname doesn't impact placekey. If such transformation is good for further address matching, we can definitely incorporate this step in the Assessor Table!

look in mls table


In [ ]:
# still for addresses with unit number 

test_noTraAbbr_mls = MLS_ADDRESS_final.copy()
# keep usaddresses in the same order as how the original string is formatted
test_noTraAbbr_mls['ColumnA'] = test_noTraAbbr_mls[['AddressNumber', 'StreetNamePreDirectional', 'StreetName',
       'StreetNamePostType','StreetNamePostDirectional','OccupancyType', 'OccupancyIdentifier']].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)
test_noTraAbbr_mls = test_noTraAbbr_mls[~test_noTraAbbr_mls['OccupancyType'].isna()][:1000]

# generate key with combined ColumnA 
column_map = {"ColumnA" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
stad_df = get_standard_df(test_noTraAbbr_mls,column_map,"US", "new york", "ny")

df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
concat_noTranAbbr_mls = pd.concat([df_placekey,test_noTraAbbr_mls.reset_index()],axis=1)


2022-05-06 10:47:34,818	INFO	Processed 1000 items
2022-05-06 10:47:34,818	INFO	Processed 1000 items
2022-05-06 10:47:34,819	INFO	Done


In [ ]:
concat_noTranAbbr_mls[concat_noTranAbbr_mls['MLS_ADDRESS']!=concat_noTranAbbr_mls['ColumnA']]

,query_id,placekey,index,MLS_ADDRESS,NAME,POSTALCODE,MLS_ADDRESS_Split,AddressNumber,StreetNamePreDirectional,StreetName,...,OccupancyType,OccupancyIdentifier,StreetNamePostDirectional,PlaceName,StateName,ZipCode,StreetNamePreType,USPSBoxGroupType,BuildingName,ColumnA
298,299,28k@627-s8r-835,383,1600 broadway apt 9g,None,10019,"({'AddressNumber': '1600', 'StreetName': 'broa...",1600,NaN,broadway,...,apt,9g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1600 broadway apt 9g
377,378,26m@627-s8h-w49,487,874 broadway apt 505,withheld,10003,"({'AddressNumber': '874', 'StreetName': 'broad...",874,NaN,broadway,...,apt,505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,874 broadway apt 505
450,451,26w@627-wc7-qzz,568,304 bowery apt 2,None,10012,"({'AddressNumber': '304', 'StreetName': 'bower...",304,NaN,bowery,...,apt,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,304 bowery apt 2
451,452,26w@627-wc7-qzz,570,304 bowery apt 2,None,10012,"({'AddressNumber': '304', 'StreetName': 'bower...",304,NaN,bowery,...,apt,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,304 bowery apt 2
462,463,28k@627-s8r-835,585,1600 broadway apt 9g,None,10019,"({'AddressNumber': '1600', 'StreetName': 'broa...",1600,NaN,broadway,...,apt,9g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1600 broadway apt 9g
537,538,25m@627-s8r-835,673,1600 broadway apt 20g,withheld,10019,"({'AddressNumber': '1600', 'StreetName': 'broa...",1600,NaN,broadway,...,apt,20g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1600 broadway apt 20g
578,579,278@627-wc7-whq,730,199 bowery apt 3c,withheld,10002,"({'AddressNumber': '199', 'StreetName': 'bower...",199,NaN,bowery,...,apt,3c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,199 bowery apt 3c
610,611,25r@627-s8r-835,770,1600 broadway apt 21e,withheld,10019,"({'AddressNumber': '1600', 'StreetName': 'broa...",1600,NaN,broadway,...,apt,21e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1600 broadway apt 21e
779,780,24n@627-s4r-4qf,995,1965 broadway apt 16h,withheld,10023,"({'AddressNumber': '1965', 'StreetName': 'broa...",1965,NaN,broadway,...,apt,16h,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1965 broadway apt 16h
787,788,23m@627-s4r-nt9,1003,2166 broadway apt 10b,opera owners inc,10024,"({'AddressNumber': '2166', 'StreetName': 'broa...",2166,NaN,broadway,...,apt,10b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2166 broadway apt 10b


In [ ]:
# no transfer => meaning that we will use original address column which has been cleaned
test_noTraAbbr_mls = MLS_ADDRESS_final.copy()
test_noTraAbbr_mls = test_noTraAbbr_mls[~test_noTraAbbr_mls['OccupancyType'].isna()][:1000]

# generate key with combined ColumnA 
column_map = {"MLS_ADDRESS" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
stad_df = get_standard_df(test_noTraAbbr_mls,column_map,"US", "new york", "ny")

df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
concat_noTranAbbr_mls = pd.concat([df_placekey,test_noTraAbbr_mls.reset_index()],axis=1)
# concat_noTranAbbr_mls

2022-05-06 10:47:41,818	INFO	Processed 1000 items
2022-05-06 10:47:41,819	INFO	Processed 1000 items
2022-05-06 10:47:41,820	INFO	Done


In [ ]:
concat_noTranAbbr_mls[concat_noTranAbbr_mls['MLS_ADDRESS']=='196 avenue of the americas apt 1c']
concat_noTranAbbr_mls


,query_id,placekey,index,MLS_ADDRESS,NAME,POSTALCODE,MLS_ADDRESS_Split,AddressNumber,StreetNamePreDirectional,StreetName,StreetNamePostType,OccupancyType,OccupancyIdentifier,StreetNamePostDirectional,PlaceName,StateName,ZipCode,StreetNamePreType,USPSBoxGroupType,BuildingName
0,1,2gh@627-wbn-k9f,0,345 w 13th st apt 4f,withheld,10014,"({'AddressNumber': '345', 'StreetNamePreDirect...",345,w,13th,st,apt,4f,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2x8@627-wc7-yn5,1,113 eldridge st apt 2b,withheld,10002,"({'AddressNumber': '113', 'StreetName': 'eldri...",113,NaN,eldridge,st,apt,2b,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,24t@627-s8h-ty9,2,50 gramercy park n apt 11a,withheld,10010,"({'AddressNumber': '50', 'StreetName': 'gramer...",50,NaN,gramercy,park,apt,11a,n,NaN,NaN,NaN,NaN,NaN,NaN
3,4,23m@627-s6m-5j9,3,69 bennett ave apt 107,withheld,10033,"({'AddressNumber': '69', 'StreetName': 'bennet...",69,NaN,bennett,ave,apt,107,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2dk@627-s8j-skf,4,245 e 24th st apt 11b,withheld,10010,"({'AddressNumber': '245', 'StreetNamePreDirect...",245,e,24th,st,apt,11b,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,26s@627-s8p-3bk,1314,45 e end ave apt 5j,withheld,10028,"({'AddressNumber': '45', 'StreetNamePreDirecti...",45,e,end,ave,apt,5j,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,997,3bh@627-s8r-89z,1315,230 w 56th st apt 64a,withheld,10019,"({'AddressNumber': '230', 'StreetNamePreDirect...",230,w,56th,st,apt,64a,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,998,26s@627-s8p-3bk,1316,45 e end ave apt 5j,withheld,10028,"({'AddressNumber': '45', 'StreetNamePreDirecti...",45,e,end,ave,apt,5j,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,999,34r@627-s4r-t9z,1317,140 w 71st st apt 6e,withheld,10023,"({'AddressNumber': '140', 'StreetNamePreDirect...",140,w,71st,st,apt,6e,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
test_TraAbbr_mls = MLS_ADDRESS_final.copy()
test_TraAbbr_mls = test_TraAbbr_mls[~test_TraAbbr_mls['OccupancyType'].isna()][:1000]

test_TraAbbr_mls['ColumnA'] = test_TraAbbr_mls[['AddressNumber', 'StreetNamePreDirectional','StreetNamePreType' ,'StreetName',
       'StreetNamePostType','StreetNamePostDirectional','OccupancyType', 'OccupancyIdentifier']].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

# test_TraAbbr_mls

In [ ]:
# generate key with combined ColumnA 
column_map = {"ColumnA" : "street_address", "POSTALCODE": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
stad_df = get_standard_df(test_TraAbbr_mls,column_map,"US", "new york", "ny")

df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
concat_TranAbbr_mls = pd.concat([df_placekey,test_TraAbbr_mls.reset_index()],axis=1)
concat_TranAbbr_mls

2022-05-06 10:47:49,610	INFO	Processed 1000 items
2022-05-06 10:47:49,611	INFO	Processed 1000 items
2022-05-06 10:47:49,611	INFO	Done


,query_id,placekey,index,MLS_ADDRESS,NAME,POSTALCODE,MLS_ADDRESS_Split,AddressNumber,StreetNamePreDirectional,StreetName,...,OccupancyType,OccupancyIdentifier,StreetNamePostDirectional,PlaceName,StateName,ZipCode,StreetNamePreType,USPSBoxGroupType,BuildingName,ColumnA
0,1,2gh@627-wbn-k9f,0,345 w 13th st apt 4f,withheld,10014,"({'AddressNumber': '345', 'StreetNamePreDirect...",345,w,13th,...,apt,4f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,345 w 13th st apt 4f
1,2,2x8@627-wc7-yn5,1,113 eldridge st apt 2b,withheld,10002,"({'AddressNumber': '113', 'StreetName': 'eldri...",113,NaN,eldridge,...,apt,2b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113 eldridge st apt 2b
2,3,24t@627-s8h-ty9,2,50 gramercy park n apt 11a,withheld,10010,"({'AddressNumber': '50', 'StreetName': 'gramer...",50,NaN,gramercy,...,apt,11a,n,NaN,NaN,NaN,NaN,NaN,NaN,50 gramercy park n apt 11a
3,4,23m@627-s6m-5j9,3,69 bennett ave apt 107,withheld,10033,"({'AddressNumber': '69', 'StreetName': 'bennet...",69,NaN,bennett,...,apt,107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69 bennett ave apt 107
4,5,2dk@627-s8j-skf,4,245 e 24th st apt 11b,withheld,10010,"({'AddressNumber': '245', 'StreetNamePreDirect...",245,e,24th,...,apt,11b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,245 e 24th st apt 11b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,26s@627-s8p-3bk,1314,45 e end ave apt 5j,withheld,10028,"({'AddressNumber': '45', 'StreetNamePreDirecti...",45,e,end,...,apt,5j,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45 e end ave apt 5j
996,997,3bh@627-s8r-89z,1315,230 w 56th st apt 64a,withheld,10019,"({'AddressNumber': '230', 'StreetNamePreDirect...",230,w,56th,...,apt,64a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230 w 56th st apt 64a
997,998,26s@627-s8p-3bk,1316,45 e end ave apt 5j,withheld,10028,"({'AddressNumber': '45', 'StreetNamePreDirecti...",45,e,end,...,apt,5j,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45 e end ave apt 5j
998,999,34r@627-s4r-t9z,1317,140 w 71st st apt 6e,withheld,10023,"({'AddressNumber': '140', 'StreetNamePreDirect...",140,w,71st,...,apt,6e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140 w 71st st apt 6e


In [ ]:
# compare if abbr matters
IfAbbrMatter_mls = pd.merge(concat_noTranAbbr_mls[['query_id','placekey','MLS_ADDRESS']],concat_TranAbbr_mls[['query_id','placekey','ColumnA']], how = 'left', right_on = ['query_id'], 
                          left_on = ['query_id'])
IfAbbrMatter_mls
IfAbbrMatter_mls[(IfAbbrMatter_mls['placekey_x']!=IfAbbrMatter_mls['placekey_y'])& (IfAbbrMatter_mls['placekey_x'].notna())]

# yay seems that there's no difference!




,query_id,placekey_x,MLS_ADDRESS,placekey_y,ColumnA


In [ ]:
# concat_noTranAbbr_mls[concat_noTranAbbr_mls['MLS_ADDRESS']=='60 sutton pl s 14gs']

Conclusion: it's okay to use converted address to generate keys!
### If unit signal exist:
1. For Accessor Table:\
there's no difference in the keys generated by converted/non-converted combined columns \
=> we can use combined column to  generate keys! it doesn't matter whether we transform those directional words/ street type or not \
=> depending on what we're gonna do in the later column-bu-column string mappings! Yay!

2. For MLS Table:\
there's no difference in the keys generated by converted combined columns vs. original column\
=> it's okay to standatdize directional words AND StreetTypes\
=> but we keep the order of ['AddressNumber', 'StreetNamePreDirectional','StreetNamePreType' ,'StreetName','StreetNamePostType','StreetNamePostDirectional','OccupancyType', 'OccupancyIdentifier']



### case2: when there is no signal

In [ ]:
# we take original tabel without abbreviation transformation
test_noTraAbbr[(test_noTraAbbr['SITUSUNITNBR'].notna())].shape # address with unit number
# shared building? 
# TODO: generate sample-> a building with mul units in the same building 




(315272, 35)

maybe get placekey id for each building (if keys being returned are the same for add. which only diff. in the unit numbers)

In [ ]:
# DT_ADDRESS_final[(DT_ADDRESS_final['SITUSUNITTYPE'].isna())&(DT_ADDRESS_final['SITUSUNITNBR'].notna())]
# concat_Abbr
noUnitNum = test_noTraAbbr[(test_noTraAbbr['SITUSUNITTYPE'].isna())&(test_noTraAbbr['SITUSUNITNBR'].notna())]

noUnitNum

# generate key with ColumnA-after string tranformation
# => no matter if we remove special characters or not, there is no missing info if we use ColumnA ( since we are using the original columns)
column_map = {"ColumnA" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
stad_df = get_standard_df(noUnitNum,column_map,"US", "new york", "ny")

df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
noUnitNum_key1 = pd.concat([df_placekey,noUnitNum.reset_index()],axis=1)


2022-05-06 10:47:56,571	INFO	Processed 745 items
2022-05-06 10:47:56,572	INFO	Done


In [ ]:
noUnitNum

,DATATREE_ADDRESS,SITUSHOUSENBR,SITUSDIRECTIONLEFT,SITUSSTREET,SITUSMODE,SITUSDIRECTIONRIGHT,SITUSUNITTYPE,SITUSUNITNBR,SITUSLATITUDE,SITUSLONGITUDE,...,AddressNumberSuffix,SubaddressType,SubaddressIdentifier,Recipient,BuildingName,StreetNamePostModifier,LandmarkName,USPSBoxType,USPSBoxID,ColumnA
32,1390 2nd ave 1,1390,None,2nd,ave,None,None,1,40.768339,-73.958287,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1390 2nd ave 1
512,260 park ave s 1,260,None,park,ave,s,None,1,40.738890,-73.987904,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,260 park ave s 1
575,31 w 31st st 2,31,w,31st,st,None,None,2,40.747391,-73.987389,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31 w 31st st 2
757,123 bank st 127,123,None,bank,st,None,None,127,40.733940,-74.005263,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123 bank st 127
1059,11 w 30th st 1e,11,w,30th,st,None,None,1e,40.746507,-73.987209,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11 w 30th st 1e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357406,31 w 31st st 1,31,w,31st,st,None,None,1,40.747391,-73.987389,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31 w 31st st 1
357536,113 prince st 2w,113,None,prince,st,None,None,2w,40.725359,-73.999581,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113 prince st 2w
357915,395 hudson st a,395,None,hudson,st,None,None,a,40.729205,-74.007773,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,395 hudson st a
358044,25 w 51st st 3,25,w,51st,st,None,None,3,40.760003,-73.978157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25 w 51st st 3


In [ ]:
test_noTraAbbr.shape

(358743, 35)

In [ ]:
# old_df.loc[old_df.product_url.isin(list(to_zero)), 'availability'] = 0
noUnitNum.loc[noUnitNum['SITUSUNITTYPE'].isna(),'SITUSUNITTYPE'] = 'apt'

In [ ]:
noUnitNum['ColumnA_new'] = noUnitNum[noUnitNum.columns[1:8]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

column_map = {"ColumnA_new" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
stad_df = get_standard_df(noUnitNum,column_map,"US", "new york", "ny")

df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
noUnitNum_key2 = pd.concat([df_placekey,noUnitNum.reset_index()],axis=1)


2022-05-06 10:48:02,305	INFO	Processed 745 items
2022-05-06 10:48:02,305	INFO	Done


In [ ]:
# final result for address which have unit number but do not have unit signals 
# => we add a signal where they are the same as the unit type occuring the most frequently 
add_unitsig = pd.merge(noUnitNum_key1[['query_id','placekey','DATATREE_ADDRESS','ColumnA']],noUnitNum_key2[['query_id','placekey','ColumnA_new']], how = 'left', right_on = ['query_id'], 
                          left_on = ['query_id'])

add_unitsig[(add_unitsig['placekey_x']!=add_unitsig['placekey_y'])].shape

# do we need to manually add a signal?


(57, 6)

57/745 -> new york

In [ ]:
add_unitsig[(add_unitsig['placekey_x']!=add_unitsig['placekey_y'])]

,query_id,placekey_x,DATATREE_ADDRESS,ColumnA,placekey_y,ColumnA_new
2,3,2c3@627-s8j-y35,31 w 31st st 2,31 w 31st st 2,26p@627-s8j-y35,31 w 31st st apt 2
12,13,2nm@627-s8r-jvz,300 e 62nd st 1,300 e 62nd st 1,2rp@627-s8r-jvz,300 e 62nd st apt 1
46,47,227@627-s8r-6zf,350 w 50th st 2c,350 w 50th st 2c,2kp@627-s8r-6zf,350 w 50th st apt 2c
47,48,2gz@627-wc7-v75,2 5th ave 3,2 5th ave 3,2hb@627-wc7-v75,2 5th ave apt 3
54,55,2g8@627-s6m-9zz,2360 amsterdam ave 1,2360 amsterdam ave 1,2h4@627-s6m-9zz,2360 amsterdam ave apt 1
74,75,NaN,12-14 w 18th st 2e,12-14 w 18th st 2e,NaN,12-14 w 18th st apt 2e
90,91,2m9@627-s4r-nt9,235 w 75th st 2,235 w 75th st 2,2qq@627-s4r-nt9,235 w 75th st apt 2
98,99,3hp@627-wbz-s3q,99 john st 1,99 john st 1,3n7@627-wbz-s3q,99 john st apt 1
103,104,2d7@627-s8j-b49,137 w 25th st 3,137 w 25th st 3,2my@627-s8j-b49,137 w 25th st apt 3
105,106,227@627-s8r-6zf,350 w 50th st 2a,350 w 50th st 2a,2cz@627-s8r-6zf,350 w 50th st apt 2a


In [ ]:
# df_DATATREE
DT_ADDRESS_final
# DT_ADDRESS_final['SITUSUNITTYPE'].value_counts()
# OccupancyType
test_noTraAbbr['SITUSUNITTYPE'].value_counts()

apt         193003
#           117527
fl            1281
ph            1145
ste            738
rm             484
ofc            149
unit           112
lbby            94
bsmt            22
broadway        14
off              6
lowr             5
bldg             4
uppr             4
pier             2
no               1
slip             1
Name: SITUSUNITTYPE, dtype: int64

In [ ]:
DT_ADDRESS_final['SITUSUNITTYPE'].value_counts()

apt         193003
#           117527
fl            1281
ph            1145
ste            738
rm             484
ofc            149
unit           112
lbby            94
bsmt            22
broadway        14
off              6
lowr             5
bldg             4
uppr             4
pier             2
no               1
slip             1
Name: SITUSUNITTYPE, dtype: int64

for addresses that have unit signal:

In [ ]:
test_DT = test_noTraAbbr[~test_noTraAbbr['SITUSUNITTYPE'].isna()][:1000]
# test_noTraAbbr[(test_noTraAbbr['SITUSUNITNBR'].notna())].shape # address with unit number


In [ ]:
# generate key with ColumnA-after string tranformation
column_map = {"ColumnA" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
stad_df = get_standard_df(test_DT,column_map,"US", "new york", "ny")

df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
concat_ColA = pd.concat([df_placekey,test_DT.reset_index()],axis=1)


2022-05-06 10:55:54,325	INFO	Processed 1000 items
2022-05-06 10:55:54,328	INFO	Processed 1000 items
2022-05-06 10:55:54,329	INFO	Done


In [ ]:
# generate key with original data - after string tranformation
column_map = {"DATATREE_ADDRESS" : "street_address", "SITUSZIP5": "postal_code"}
place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
stad_df = get_standard_df(test_DT,column_map,"US", "new york", "ny")

df_placekey = get_placekey_df(pk_api,stad_df,place_keys)
concat_Orig =pd.concat([df_placekey,test_DT.reset_index()],axis=1)


2022-05-06 10:57:12,719	INFO	Processed 1000 items
2022-05-06 10:57:12,722	INFO	Processed 1000 items
2022-05-06 10:57:12,723	INFO	Done


In [ ]:
concat_ColA[['query_id','error','placekey','DATATREE_ADDRESS','ColumnA']]

,query_id,error,placekey,DATATREE_ADDRESS,ColumnA
0,1,NaN,2p3@627-s8j-yvz,308 w 30th st apt 9b,308 w 30th st apt 9b
1,2,Invalid address,NaN,138-140 w 124th st # 5a,138-140 w 124th st # 5a
2,3,NaN,2h6@627-wbn-k9f,345 w 14th st apt 4b,345 w 14th st apt 4b
3,4,NaN,22f@627-s8j-d5f,7 w 14th st # 1120,7 w 14th st # 1120
4,5,NaN,2vj@627-s4n-2zf,340 e 74th st apt 5k,340 e 74th st apt 5k
...,...,...,...,...,...
995,996,NaN,225@627-s4n-66k,1334 3rd ave # 16e,1334 3rd ave # 16e
996,997,NaN,23m@627-s8r-fj9,30 rockefeller plz # t53-1,30 rockefeller plz # t53-1
997,998,NaN,224@627-s8n-y9z,509 e 77th st # 2-0-5p,509 e 77th st # 2-0-5p
998,999,Invalid address,NaN,36 park row # 6e,36 park row # 6e


In [ ]:
concat_Orig[['query_id','error','placekey','DATATREE_ADDRESS','ColumnA']]

,query_id,error,placekey,DATATREE_ADDRESS,ColumnA
0,1,NaN,2p3@627-s8j-yvz,308 w 30th st apt 9b,308 w 30th st apt 9b
1,2,Invalid address,NaN,138-140 w 124th st # 5a,138-140 w 124th st # 5a
2,3,NaN,2h6@627-wbn-k9f,345 w 14th st apt 4b,345 w 14th st apt 4b
3,4,NaN,22f@627-s8j-d5f,7 w 14th st # 1120,7 w 14th st # 1120
4,5,NaN,2vj@627-s4n-2zf,340 e 74th st apt 5k,340 e 74th st apt 5k
...,...,...,...,...,...
995,996,NaN,225@627-s4n-66k,1334 3rd ave # 16e,1334 3rd ave # 16e
996,997,NaN,23m@627-s8r-fj9,30 rockefeller plz # t53-1,30 rockefeller plz # t53-1
997,998,NaN,224@627-s8n-y9z,509 e 77th st # 2-0-5p,509 e 77th st # 2-0-5p
998,999,Invalid address,NaN,36 park row # 6e,36 park row # 6e
